In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [2]:
model_name = "/root/autodl-fs/models/TheBloke/Mixtral-8x7B-Instruct-v0___1-GPTQ/"

# 加载分词器和模型
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
llm = AutoModelForCausalLM.from_pretrained(
    model_name, 
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16
    )

/root/miniconda3/envs/coh/lib/python3.9/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:411: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, qweight, scales, qzeros, g_idx, bits, maxq):
/root/miniconda3/envs/coh/lib/python3.9/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:419: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/root/miniconda3/envs/coh/lib/python3.9/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)
CUDA extension not installed.
CUDA extension not installed.
`loss_type=None` was set in the config but it is unrecognised.Usi

In [4]:
from transformers import GenerationConfig

from coh.prompts import HIS_TO_ANSWER

generation_config = GenerationConfig(
    max_new_tokens=8000,
    top_p=1.0,
    temperature=0.0,
    do_sample=False, # Set to False for greedy decoding (temperature=0.0 implies this)
    pad_token_id=tokenizer.eos_token_id, # Good practice for batching or if model generates past max_length
    eos_token_id=tokenizer.eos_token_id # Stop generation when EOS token is produced
)

input_text = HIS_TO_ANSWER.format(
    histories="""
Government_(Nigeria), Engage_in_diplomatic_cooperation with, Independent_Corrupt_Practices_Commission, on the 339th day; Independent_Corrupt_Practices_Commission, Arrest_or_detain_or_charge_with_legal_action to, Citizen_(Nigeria), on the 308th day;
Government_(Nigeria), Criticize_or_denounce, Boko_Haram, on the 337th day;
Boko_Haram, Use_conventional_military_force to, Citizen_(Nigeria), on the 336th day;
Government_(Nigeria), Threaten, Education_(Nigeria), on the 337th day;
Education_(Nigeria), Make_statement to, Muslim_(Nigeria), on the 332nd day;
Government_(Nigeria), Make_optimistic_comment on, Citizen_(Nigeria), on the 336th day;
Citizen_(Nigeria), Make_an_appeal_or_request to, Member_of_the_Judiciary_(Nigeria), on the 331st day;
· · · · · ·
    """,
    query="""
Government_(Nigeria), Make_an_appeal_or_request to, whom, on the 340th day?
    """
)

# Encode the input
inputs = tokenizer(input_text, return_tensors="pt").to(llm.device)

output_sequences = llm.generate(
    **inputs,
    generation_config=generation_config
)

generated_ids = output_sequences[0, inputs.input_ids.shape[1]:] # Slice to get only new tokens
generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)

print(f"Original Prompt: {input_text}")
print(f"Generated Text: {generated_text}")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
